In [2]:
import pandas as pd
import os

In [4]:
test_df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-sedelkin/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_1.csv', 
                sep=';', 
                parse_dates=['ACT_DTTM'])

In [8]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-sedelkin/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/'

In [9]:
tm_sales = pd.DataFrame()
for i in os.listdir(path)[1:]:
    df = pd.read_csv(path + i, sep=';', parse_dates=['ACT_DTTM'])
    tm_sales = tm_sales.append(df)

In [10]:
tm_sales = tm_sales.dropna()

In [11]:
def add_id(i):
    if str(i).startswith('id'):
        return i
    else:
        return 'id' + str(i)

In [12]:
tm_sales['SUBS_ID'] = tm_sales.SUBS_ID.apply(add_id)

In [31]:
tm_sales.dtypes

SUBS_ID              object
FILIAL_ID             int64
PROD_ID               int64
ACT_DTTM     datetime64[ns]
dtype: object

In [16]:
os.listdir(path)

['prod_activations_logs.csv',
 'tm_sales_1.csv',
 'tm_sales_2.csv',
 'tm_sales_3.csv']

In [78]:
prod_activations = pd.read_csv(path + 'prod_activations_logs.csv', sep=';')

In [88]:
prod_activations.START_DTTM = pd.to_datetime(prod_activations.START_DTTM, format='%d-%m-%Y %H:%M')
prod_activations.END_DTTM = pd.to_datetime(prod_activations.END_DTTM, format='%d-%m-%Y %H:%M')

In [34]:
prod_activations.START_DTTM = prod_activations.START_DTTM.astype('datetime64[ns]')
prod_activations.END_DTTM = prod_activations.END_DTTM.astype('datetime64[ns]')

In [89]:
prod_activations.head(15)

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


In [90]:
tm_join = tm_sales.merge(prod_activations, on='SUBS_ID')

In [91]:
tm_join['time_delta'] = tm_join.END_DTTM - tm_join.START_DTTM
tm_join.head(10)

,SUBS_ID,FILIAL_ID,PROD_ID_x,ACT_DTTM,PROD_ID_y,START_DTTM,END_DTTM,time_delta
0,id4651830,1,1954,2020-03-20 14:59:00,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,2020-03-19 13:00:00,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,2020-03-17 10:17:00,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,2020-03-16 16:28:00,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,2020-03-15 14:21:00,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [92]:
target_delta = pd.Timedelta(5, 'm')
target_delta

Timedelta('0 days 00:05:00')

In [93]:
tm_join[tm_join.time_delta < target_delta]

,SUBS_ID,FILIAL_ID,PROD_ID_x,ACT_DTTM,PROD_ID_y,START_DTTM,END_DTTM,time_delta
1,id7646509,5,6431,2020-03-19 13:00:00,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00


In [94]:
tm_join.drop(tm_join[tm_join.time_delta <= target_delta].index, inplace=True)

In [97]:
print(tm_join.sort_values(by='SUBS_ID').SUBS_ID)

3    id2185490
0    id4651830
2    id5416547
4    id7642700
Name: SUBS_ID, dtype: object
